#Aula 1

In [3]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.9 MB/s eta 0:00:00


Importação da API Key

In [4]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Gemini

In [12]:
lln = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    api_key=GOOGLE_API_KEY
)

Teste básico do modelo com uma pergunta

In [13]:
resp_teste = lln.invoke('Quem é voce?')
print(resp_teste.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [14]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
)

Definição do Modelo de Saída com Pydantic

In [15]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO']
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

Criação do LLM para Classificação e da Chain

In [16]:
lln_triagem = ChatGoogleGenerativeAI (
    model='gemini-2.5-flash',
    temperature=0,
    api_key=GOOGLE_API_KEY
)

Importa mensagens e cria a chain com saída estruturada.

In [17]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = lln_triagem.with_structured_output(TriagemOut)

def triagem(mensagem : str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()

Testes do Agente

In [18]:
testes =  ["Posso reembolsar a internet?",
           "Quero ter mais 5 dias remoto, Como faco?",
           "Posso reembolsar cursos ou treinamentos do Alura",
           "Quantas capivaras tem no Rio Pinheiros?"]

for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")